In [ ]:
# https://github.com/uclnlp/emoji2vec/tree/master

In [1]:
from transformers import AutoModel
from numpy.linalg import norm

cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-zh', trust_remote_code=True) # trust_remote_code is needed to use the encode method
embeddings = model.encode(['How is the weather today?', '今天天气怎么样?'])
print(cos_sim(embeddings[0], embeddings[1]))

/data/clipx/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


0.7860602


In [44]:
emoji_des_fn = "emoji_table0.txt"
emoji_table = [l.strip().split("\t") for l in open(emoji_des_fn).readlines()]
emoji_des = {}
for d,k in emoji_table:
    d = d.lower()
    if k in emoji_des:
        emoji_des[k].append(d)
    else:
        emoji_des[k] = [d]
# emoji_des = {k:", ".join(v) for k,v in emoji_des.items()}
emoji_des0 = emoji_des

In [45]:
def decode_emoji(emoji_code):
    return chr(int(emoji_code[2:], 16))
emoji_des0 = {decode_emoji(k): v for k, v in emoji_des0.items()}

In [46]:
emoji_des_fn = "emoji_table.txt"
emoji_table = [l.strip().split("\t") for l in open(emoji_des_fn).readlines()]
emoji_des = {}
for d,k in emoji_table:
    d = d.lower()
    if k in emoji_des:
        emoji_des[k].append(d)
    else:
        emoji_des[k] = [d]
# emoji_des = {k:", ".join(v) for k,v in emoji_des.items()}
emoji_des1 = emoji_des


In [47]:
len(emoji_des0), len(emoji_des1)

(1012, 1661)

In [49]:
def merge_lists(list1, list2):
    d = dict.fromkeys(list1)
    d.update(dict.fromkeys(list2))
    return list(d.keys())

emoji_des = emoji_des0
for k,v in emoji_des1.items():
    if k not in emoji_des:
        emoji_des[k] = v
    else:
        emoji_des[k] = merge_lists(emoji_des[k], v)


In [51]:
len(emoji_des)

1735

In [53]:
emoji_des = {k:", ".join(v) for k,v in emoji_des.items()}

In [55]:
# emoji_des0['🕟'],
emoji_des['🏋']

'weight lifter, weightlifting'

In [56]:
emoji_embeddings = {k:model.encode(v) for k,v in emoji_des.items()}

In [57]:
len(emoji_embeddings)

1735

In [58]:
import pickle
# save emoji_embeddings
pickle.dump(emoji_embeddings, open('emoji_embeddings.pkl', 'wb'))

# load emoji_embeddings
emoji_embeddings = pickle.load(open('emoji_embeddings.pkl', 'rb'))

In [59]:
nomred_emoji_embeddings = {k:v/norm(v) for k,v in emoji_embeddings.items()}
pickle.dump(nomred_emoji_embeddings, open('emoji_normed_embeddings.pkl', 'wb'))
emoji_embeddings = pickle.load(open('emoji_normed_embeddings.pkl', 'rb'))

In [60]:
import numpy as np
emoji_list = np.array(list(emoji_embeddings.keys()))

# emoji_list = np.array([decode_emoji(emoji_code) for emoji_code in emoji_code_list])

In [61]:
emoji_Fs = np.stack(list(emoji_embeddings.values()))

In [62]:
# dump emoji_Fs use numpy

np.save("emoji.npy", {"emoji_features": emoji_Fs, "emoji_list": emoji_list})
    


emoji = np.load('emoji.npy', allow_pickle=True).item()
emoji_features = emoji['emoji_features']
